In [1]:
# Using code from
# https://www.annytab.com/a-star-search-algorithm-in-python/
# Additional explanations are needed
# Magic Methods - https://www.tutorialsteacher.com/python/magic-methods-in-python

In [2]:
import numpy as np
import pandas as pd

In [2]:
# Create a graph class
class Graph:
    # The class needs to be initialised
    # self is used to refer to the instance of the class
    # This is similar to using the 'this' in other languages
    # There is a slight difference as self must be the first parameter passed
    # __init__ is reserved as it is the constructor method
    # The graph will be directed so set the parameter to true
    # If no dictionary is given an empty one will be used
    def __init__(self, graph_dict=None, directed=True):
        # Sets the graph_dict to the instance or an empty dictionary
        self.graph_dict = graph_dict or {}
        self.directed = directed
        
        # If this is not a directed graph then set to undirected 
        # by calling the make_undirected method (From below)
        if not directed:
            self.make_undirected()
    
    # Make an undirected graph
    def make_undirected(self):
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.graph_dict.setdefault(b, {})[a] = dist
    
    # Add a link from A and B of given distance
    # Also add the inverse link if the graph is undirected
    def connect(self, A, B, distance=1):
        self.graph_dict.setdefault(A, {})[B] = distance
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = distance
    
    # Get neighbours or a neighbour
    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)

    # Return a list of nodes in the graph
    def nodes(self):
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        # This will return a set that contains all items from both sets
        # Duplicates are excluded
        nodes = s1.union(s2)
        return list(nodes)
        

In [15]:
# Create a node class which represents a node
class Node:
    # Initialise the Node class
    def __init__(self, name:str, parent:str):
        self.name = name
        self.parent = parent
        # Initialise values to 0
        # The distance to the start node
        self.g = 0
        # The distance to goal node
        self.h = 0
        # Total Cost
        self.f = 0
    
    # Compare nodes __eq__ compares 2 objects by their values
    def __eq__(self, other):
        return self.name == other.name
    
    # Sort the nodes, lt is the less than operator
    def __lt__(self, other):
        return self.f < other.f
    
    # Print node, __repr__ represents class objects as strings
    def __repr__(self):
        return ('({0},{1})'.format(self.name, self.f))
    
# The a* star search method
def astar_search(graph, heuristics, start, end):
    # Create 2 lists, one for open and one for closed nodes
    open = []
    closed = []

    # Create a start node and also a goal node
    start_node = Node(start, None)
    goal_node = Node(end, None)

    # Add a start node
    open.append(start_node)

    # Loop through the open list until it is empty
    while len(open) > 0:
        # Sort the list so that you get the lowest cost node first
        # Sort() sorts the list in ascending order first
        open.sort()

        # Get the node that has the lowest cost
        current_node = open.pop(0)

        # Then add the popped node to the closed list
        closed.append(current_node)

        # Check to see if the goal has been reached then return the path
        if current_node == goal_node:
            path = []
            while current_node != start_node:
                path.append(current_node.name + ': ' + str(current_node.g))
                current_node = current_node.parent
            path.append(start_node.name + ': ' + str(start_node.g))
            # Returns the reversed path
            return path[::-1]

        # Get the neighbours
        neighbours = graph.get(current_node.name)

        # Loop through the neighbours
        for key, value in neighbours.items():
            # Creates a neighbour node
            neighbour = Node(key, current_node)

            # Checks to see if the neighbour is in rhe closed list
            if(neighbour in closed):
                continue

            # Calculate the full path cost
            neighbour.g = current_node.g + graph.get(current_node.name, neighbour.name)
            neighbour.h = heuristics.get(neighbour.name)
            neighbour.f = neighbour.g
            
            # Check to see if the neighbour is in the open list
            # Also check to see if it has a lower cost value
            if(add_to_open(open, neighbour) == True):
                # If everything is ok then add the neighbour to the open list
                open.append(neighbour)

    # Return None, no path is found
    return None
    
# Check if a neighbor should be added to open list
def add_to_open(open, neighbour):
    for node in open:
        if (neighbour == node and neighbour.f > node.f):
            return False
    return True

# The main method is the entry point into the program
# A lot of languages have this explicitly as the program entry point
def main():
    # Create a graph
    graph = Graph()

    # Create the graph connections with the appropriate distances
    graph.connect('Neamt', 'Iasi', 87)
    graph.connect('Iasi', 'Vaslui', 92)
    graph.connect('Vaslui', 'Urziceni', 142)
    graph.connect('Urziceni', 'Hirsova', 98)
    graph.connect('Urziceni', 'Bucharest', 85)
    graph.connect('Hirsova', 'Eforie', 86)
    graph.connect('Bucharest', 'Giurgiu', 90)
    graph.connect('Bucharest', 'Fargas', 211)
    graph.connect('Bucharest', 'Pitesti', 101)
    graph.connect('Pitesti', 'Craiova', 138)
    graph.connect('Pitesti', 'Rimnicu Vilcea', 97)
    graph.connect('Craiova', 'Rimnicu Vilcea', 146)
    graph.connect('Fargas', 'Sibiu', 99)
    graph.connect('Sibiu', 'Rimnicu Vilcea', 80)
    graph.connect('Sibiu', 'Oradea', 151)
    graph.connect('Ordea', 'Zerind', 71)
    graph.connect('Zerind', 'Arad', 75)
    graph.connect('Arad', 'Sibiu', 140)
    graph.connect('Arad', 'Timisoara', 118)
    graph.connect('Timisoara', 'Lugoj', 111)
    graph.connect('Lugoj', 'Mehadia', 70)
    graph.connect('Mehadia', 'Drobeta', 75)
    graph.connect('Drobeta', 'Craiova', 120)

    # Make the graph undirected and create symmetric connections
    graph.make_undirected()

    # Create heuristics
    heuristics = {}
    heuristics['Neamt'] = 1
    heuristics['Iasi'] = 1
    heuristics['Vaslui'] = 1
    heuristics['Urziceni'] = 1
    heuristics['Hirsova'] = 1
    heuristics['Bucharest'] = 1
    heuristics['Giurgiu'] = 1
    heuristics['Faragas'] = 1
    heuristics['Pitesti'] = 1
    heuristics['Craiova'] = 1
    heuristics['Rimnicu Vilcea'] = 1
    heuristics['Sibiu'] = 1
    heuristics['Oradea'] = 1
    heuristics['Zerind'] = 1
    heuristics['Arad'] = 1
    heuristics['Timisoara'] = 1
    heuristics['Lugoj'] = 1
    heuristics['Mehadia'] = 1
    heuristics['Droheta'] = 1

    # Run the search algorithm
    path = astar_search(graph, heuristics, 'Oradea', 'Vaslui')
    print(path)
    print()

# Tells Python to run the main method
# https://www.freecodecamp.org/news/if-name-main-python-example/
if __name__ == "__main__": main()